# Chapter 2: Multi-armed Bandits

### Exercise 2.1

Q: In $\varepsilon$-greedy action selection, for the case of two actions and $\varepsilon = 0.5$, what is the probability that the greedy action is selected?

**A**: In $\varepsilon$-greedy action selection, the greedy action be selected in one of two ways:
1. With probability $1 - \varepsilon$ it is chosen as the exploitation action;
2. With probability $\tfrac{\varepsilon}{|\mathcal{A}|}$ (where $\mathcal{A}$ is the action space), it could be chosen at random as an exploration action.

These are mutually exclusive events, so the total probability of the greedy action being chosen is

$$1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|}.$$

In the question, $|\mathcal{A}| = 2$ and $\varepsilon = 0.5$. Hence the greedy action is chosen with probability

$$1 - 0.5 + \frac{0.5}{2} = 0.75.$$

### Exercise 2.2

Q: Consider a $k$-armed bandit problem with $k=4$ actions, denoted 1, 2, 3 and 4. Consider applying to this problem a bandit algorithm using $\varepsilon$-greedy action selection, sample-avarage action-value estimates, and initial estimates of $Q_1(a) = 0$ for all $a$. Suppose the initial sequence of actions and rewards is

$$i$$ | $$A_i$$ | $$R_i$$
:-: | :-: | :-:
 1 | 1 | 1 
 2 | 2 | 1
 3 | 2 | 2
 4 | 2 | 2
 5 | 3 | 0

On some of these times steps the $\varepsilon$ case may have occurred, causing an action to be selected at random. On which time steps did this definitely occur? On which time steps could this possibly have occurred?

A: We can manually calculate the action values as follows:

In [1]:
import pandas as pd

# Initial data
action_space = [1, 2, 3, 4]
actions = [1, 2, 2, 2, 3]
rewards = [1, 1, 2, 2, 0]

# Set up initial Q values and action counts N for i=1
Q = {a: 0 for a in action_space}
N = {a: 0 for a in action_space}

# Iteratively update N and Q for remaining time steps and save Q history
Q_history = []
for i, (A, R) in enumerate(zip(actions, rewards), 1):
    Q_history.append(Q.copy())
    N[A] += 1
    Q[A] = Q[A] + (1.0 / N[A]) * (R - Q[A])

# Display table of action values
(
    pd.DataFrame.from_records(Q_history, index=range(1, len(actions) + 1))
    .rename(lambda a:f"$$Q({a})$$", axis=1)
    .rename_axis("$$i$$", axis=0)
)

,$$Q(1)$$,$$Q(2)$$,$$Q(3)$$,$$Q(4)$$
$$i$$,,,,
1,0.0,0.000000,0,0
2,1.0,0.000000,0,0
3,1.0,1.000000,0,0
4,1.0,1.500000,0,0
5,1.0,1.666667,0,0


From these we can determine whether a greedy action was chosen at each step:

In [2]:
chosen_action_values = [Q[A] for Q, A in zip(Q_history, actions)]
greedy_action_values = [max(Q.values()) for Q in Q_history]
definitely_exploration = [c != g for c, g in zip(chosen_action_values, greedy_action_values)]

# Display results as a table
pd.DataFrame(
    {
        "$$\max_{a \in \mathcal{A}} Q_i(a)$$": greedy_action_values,
        "$$Q_i(A_i)$$": chosen_action_values,
        "Definitely exploration?": definitely_exploration,
    },
    index=pd.Index(range(1, len(actions) + 1), name="$$i$$"),
)

,$$\max_{a \in \mathcal{A}} Q_i(a)$$,$$Q_i(A_i)$$,Definitely exploration?
$$i$$,,,
1,0.000000,0.0,False
2,1.000000,0.0,True
3,1.000000,1.0,False
4,1.500000,1.5,False
5,1.666667,0.0,True


So we can see that at timesteps $i=2$ and $i=5$, the chosen action was **not** the greedy action and hence exploration (the "$\varepsilon$ case") definitely happened. For all other time steps, we do not know: perhaps the greedy action was chosen "on purpose", i.e. as an exploitation action, or perhaps it was still chosen as an exploration action.